# MLflowのtensorサポート

本ノートブックでは、MLflowにおけるtensorサポートをデモします。

<table>
  <tr><th>作者</th><th>Databricks Japan</th></tr>
  <tr><td>日付</td><td>2021/06/29</td></tr>
  <tr><td>バージョン</td><td>1.0</td></tr>
  <tr><td>クラスター</td><td>8.3ML</td></tr>
</table>
<img style="margin-top:25px;" src="https://jixjiadatabricks.blob.core.windows.net/images/databricks-logo-small-new.png" width="140">

[MLflowでTensorの入力をサポートしました \- Qiita](https://qiita.com/taka_yayoi/items/3e439dc5df7257fd41db)

## 準備

In [0]:
# このサンプルを実行するにはMLflow >=1.14.0 が必要となります
import tensorflow.keras
from tensorflow.keras.layers import Dense, Flatten, Dropout
import numpy as np
import mlflow
import mlflow.keras
from mlflow.models.signature import infer_signature

# トレーニングデータを準備しましょう
(train_X, train_Y), (test_X, test_Y) = tensorflow.keras.datasets.mnist.load_data()
trainX, testX = train_X / 255.0, test_X / 255.0
trainY = tensorflow.keras.utils.to_categorical(train_Y)
testY = tensorflow.keras.utils.to_categorical(test_Y)

In [0]:
assert mlflow.__version__>="1.14.0", "This notebook requires MLflow>=1.14.0"

## モデルの定義

In [0]:
# モデルを定義しましょう
model = tensorflow.keras.models.Sequential(
    [
      Flatten(),
      Dense(128, activation="relu", name="layer1"),
      Dropout(0.2),
      Dense(10, activation='softmax')
    ]
)
opt = tensorflow.keras.optimizers.SGD(lr=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

## トレーニング

In [0]:
# モデルをfitさせましょう
model.fit(trainX, trainY, epochs=2, batch_size=32, validation_data=(testX, testY))

Epoch 1/2
 1/1875 [..............................] - ETA: 12:35 - loss: 2.2436 - accuracy: 0.1562WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0015s vs `on_train_batch_end` time: 0.0026s). Check your callbacks.
 24/1875 [..............................] - ETA: 4s - loss: 2.0914 - accuracy: 0.2524  59/1875 [..............................] - ETA: 3s - loss: 1.7728 - accuracy: 0.4090 94/1875 [>.............................] - ETA: 2s - loss: 1.5554 - accuracy: 0.4948 125/1875 [=>............................] - ETA: 2s - loss: 1.4182 - accuracy: 0.5451 155/1875 [=>............................] - ETA: 2s - loss: 1.3185 - accuracy: 0.5803 187/1875 [=>............................] - ETA: 2s - loss: 1.2354 - accuracy: 0.6091 221/1875 [==>...........................] - ETA: 2s - loss: 1.1645 - accuracy: 0.6333 255/1875 [===>..........................] - ETA: 2s - loss: 1.1066 - accuracy: 0.6529 290/1875 [===>..........................] - ETA: 2s - loss: 1.0566 - accuracy: 0.6696 322/1875 [====>.........................] - ETA: 2s - loss: 1.0172 - accuracy: 0.6827 357/1875 [====>.........................] - ETA: 2s - loss: 0.9797 - accuracy: 0.6953 392/1875 [=====>........................] - ETA: 2s - loss: 0.9469 - accuracy: 0.7062 427/1875 [=====>........................] - ETA: 2s - loss: 0.9178 - accuracy: 0.7159 462/1875 [======>.......................] - ETA: 2s - loss: 0.8917 - accuracy: 0.7245 497/1875 [======>.......................] - ETA: 2s - loss: 0.8680 - accuracy: 0.7324 528/1875 [=======>......................] - ETA: 2s - loss: 0.8489 - accuracy: 0.7387 563/1875 [========>.....................] - ETA: 2s - loss: 0.8291 - accuracy: 0.7452 598/1875 [========>.....................] - ETA: 1s - loss: 0.8109 - accuracy: 0.7511 633/1875 [=========>....................] - ETA: 1s - loss: 0.7940 - accuracy: 0.7567 668/1875 [=========>....................] - ETA: 1s - loss: 0.7783 - accuracy: 0.7618 703/1875 [==========>...................] - ETA: 1s - loss: 0.7636 - accuracy: 0.7666 733/18

## シグネチャの作成

In [0]:
# MLflowモデルレジストリに格納するためにtensor入力を用いてモデルのシグネチャを作成します
signature = infer_signature(testX, model.predict(testX))
# どのように見えるかを確認します
print(signature)

inputs: 
 [Tensor('float64', (-1, 28, 28))]
outputs: 
 [Tensor('float32', (-1, 10))]

## 入力サンプルの作成

In [0]:
# MLflowモデルレジストリに格納する入力サンプルを作成します
input_example = np.expand_dims(trainX[0], axis=0)

## モデルをMLflowモデルレジストリに登録

In [0]:
# モデルをMLflowモデルレジストリに登録しましょう
model_name = 'mnist_example'
registered_model_name = "tensor-blog-post"
mlflow.keras.log_model(model, model_name, signature=signature, input_example=input_example, registered_model_name=registered_model_name)

INFO:tensorflow:Assets written to: /tmp/tmprgp7cw_g/model/data/model/assets
Successfully registered model 'tensor-blog-post'.
2021/06/29 01:08:37 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: tensor-blog-post, version 1
Created version '1' of model 'tensor-blog-post'.

## モデルのロード、予測

In [0]:
# モデルをロードしてサンプルの予測を実行しましょう
model_version = "1"
loaded_model = mlflow.pyfunc.load_model(f"models:/{registered_model_name}/{model_version}")
loaded_model.predict(input_example)

/databricks/python/lib/python3.8/site-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
 warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '
Out[16]: array([[2.3686067e-05, 3.0581126e-05, 5.0218968e-04, 6.4691156e-02,
 1.6546832e-09, 9.3454641e-01, 2.2379486e-06, 4.5409055e-05,
 4.9554299e-05, 1.0872682e-04]], dtype=float32)

# END